<h1 align="center" style="background-color:#616161;color:white">Outlier Analysis & Cleanup</h1>

<h3>Summary</h3>

* Two types of analysis was conducted
    - Daily LIstening Habits: Analysis of the number of unique tracks vs. num. of plays a user seen on a daily basis
    - Histogram of the time period in between song plays
    

<h3 style="background-color:#616161;color:white">0. Code setup</h3>

In [99]:
import os
import sys
import pandas as pd
import numpy as np
import datetime
import csv
import json
import sqlite3
from pathlib import Path
from bokeh.charts import *
from bokeh.sampledata.autompg import autompg as df
from bokeh.models import HoverTool, BoxSelectTool,BoxZoomTool,ResetTool
from bokeh.models import NumeralTickFormatter

In [38]:
## Parameters you can change

# Abs path to settings file
root = "C:/DS/Github/MusicRecommendation"  # BA, Windows
i = datetime.datetime.now()
outputPath = root + "/4_preliminaryanalysis/outputs/%s_%s_%s/" % (i.day, i.month, i.year) 

## Finish setting up
os.chdir(root)
## Add the prelim module
fPath = root + "/1_codebase"
if fPath not in sys.path: sys.path.append(fPath)
from codebase import *

settingsDict =  loadSettings()

# Load data from database
dbPath = root + settingsDict['dbPath']

#--------------------- Functions ----------------------------------#
def getRandomUsers(maxUsers):
    db = sqlite3.connect(dbPath)
    SQStr ="SELECT userID FROM tblUser ORDER BY RANDOM() LIMIT " + maxUsers

def exportToCSV(cursor, fPath):
    
    # Check output folder exists
    _ensurePathExists(fPath)
    
    with open(fPath, "w", newline='') as csv_file:  # Python 3 version    
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([i[0] for i in cursor.description]) # write headers
        csv_writer.writerows(cursor)


def _ensurePathExists(fPath):
    path = Path(fPath)
    path.parent.mkdir(parents=True, exist_ok=True) 

In [39]:
hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("userID", "@user"),
            ("PlayedTimestamp","@PlayedTimestamp"),
        ]
    )
TOOLS = [BoxSelectTool(),BoxZoomTool(), ResetTool(),hover]

<h3 style="background-color:#616161;color:white">1. Generate CSV exports for analysis in visualization software</h3>

In [32]:
con = sqlite3.connect(dbPath)
cur = con.cursor()

sqlStr ='SELECT Cast(substr(userID,-5) as integer) as user,date(PlayedTimestamp) as PlayedTimeStamp ,count(*) as NumOfPlays, count(Distinct trackID) as NumOfTracks from tblInputData group by userID, date(PlayedTimestamp) ORDER BY NumOfPlays;'

# Export to CSV
cur.execute(sqlStr)
exportToCSV(cur,outputPath + 'dataset1.csv')

con.close()

<h3 style="background-color:#616161;color:white">2. Basic analysis</h3>

In [41]:
con = sqlite3.connect(dbPath)
sqlStr ='SELECT Cast(substr(userID,-5) as integer) as user,date(PlayedTimestamp) as PlayedTimeStamp ,count(*) as NumOfPlays, count(Distinct trackID) as NumOfTracks from tblInputData group by userID, date(PlayedTimestamp) ORDER BY NumOfPlays;'

# Load into Pandas
res = pd.read_sql_query(sqlStr, con)
con.close()

# Change data types
res['user'] = res['user'].astype('str')
res['PlayedTimeStamp'] =  pd.to_datetime(res['PlayedTimeStamp'])
#res.dtypes

In [142]:
res['PlayedTimeStamp'].describe()

count                  391475
unique                   1589
top       2009-04-27 00:00:00
freq                      439
first     2005-02-14 00:00:00
last      2013-09-29 00:00:00
Name: PlayedTimeStamp, dtype: object

* The date range is from 27th April 2009 to 29th Sept 2013 

In [125]:
res['user'].describe()

count     391475
unique       992
top          833
freq        1457
Name: user, dtype: object

* There are 992 unique users. 

In [141]:
res.describe(percentiles = [.5, .95, .99])

,NumOfPlays,NumOfTracks
count,391475.000000,391475.000000
mean,48.919773,35.828385
std,57.588075,42.043862
min,1.000000,0.000000
50%,31.000000,23.000000
95%,153.000000,112.000000
99%,295.000000,205.000000
max,2862.000000,1520.000000


* The average number of daily plays was 48, and the average number of unique daily tracks was 35
* The higest number of daily plays was a very large <b>2862</b>
* The 99th percentile was only 295, so perhaps this makes a good cut-off point

In [145]:
qNumOfPlays = 295

res[(res['NumOfPlays'] > qNumOfPlays)].user.nunique()

# If you wish to drill down further use this:
#res[(res['NumOfPlays'] > qNumOfPlays)].groupby(['user']).count()

253

* We would be excluding 253 users if we did this - which is a large portion of our 992 unique users.

In [155]:
res.groupby(['user']).mean().describe([.5, .95, .99])

,NumOfPlays,NumOfTracks
count,992.000000,992.000000
mean,45.911395,33.153739
std,36.674290,25.153083
min,1.000000,0.666667
50%,35.920913,27.071905
95%,108.196133,78.911361
99%,208.585319,142.503486
max,337.670588,243.470703


* When we take the average tracks played on a daily basis by user we get a 99th percentile of 208 with the max average being 337
* The question at this stage is whether we exclude any days where the number of tracks played by a user exceeded a certain threshold (say 295). 
* Our analysis suggests that a large portion of users (253 our of 992) did have such excessive plays therefore exlcuding this many users is not an option
* If we average out across thee days then we find the data looks more normal - the max average was 337 tracks, still large but within the bounds of reality.
* Of course why there are some days with excessively high track plays is a mystery. Further analysis did not show any obvious patterns other than user 8 who appeared to be particularly excessive number of plays 

In [200]:
tmp=res[(res['NumOfPlays'] > 800)].groupby(['user']).count()
tmp

,PlayedTimeStamp,NumOfPlays,NumOfTracks
user,,,
155,1,1,1
199,2,2,2
262,1,1,1
554,1,1,1
666,1,1,1
791,2,2,2
8,17,17,17


In [217]:
#res[(res['user'] =='8') & (res['NumOfPlays'] > 500)]
res[(res['user'] =='8')].mean()

NumOfPlays     315.550847
NumOfTracks     20.084746
dtype: float64

<h3 style="background-color:#616161;color:white">2. Interval time</h3>

In [ ]:
con = sqlite3.connect(dbPath)
sqlStr ='Select userID, (SecsSincePrevPlay/60) as MinsSincePrevPlay, PlayedTimeStamp, historyID from tblMain order by userID, historyID'
res = pd.read_sql_query(sqlStr, con)
con.close()

res.head(10)
# Change data types
#res['userID'] = res['userID'].astype('str')
#res['PlayedTimeStamp'] =  pd.to_datetime(res['PlayedTimeStamp'])

In [ ]:
# Change data types
#res['userID'] = res['userID'].astype('str')
#res['PlayedTimeStamp'] =  pd.to_datetime(res['PlayedTimeStamp'])

res.head(1)

<h3 style="background-color:#616161;color:white">3. Custom Analysis</h3>

<h3 style="background-color:#616161;color:white">END</h3>